<a href="https://colab.research.google.com/github/UN-GCPDS/python-gcpds.EEG_Tensorflow_models/blob/main/Examples/GIGA17/dmtl_bci_giga.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U git+https://github.com/UN-GCPDS/python-gcpds.EEG_Tensorflow_models.git >/dev/null

In [ ]:
!pip install mne==0.23

In [ ]:
from EEG_Tensorflow_models.Utils.LoadData import load_dataset
from EEG_Tensorflow_models.Utils.Callbacks import get_callbacks
from EEG_Tensorflow_models.Utils.TrainingModels import get_loss,get_model,get_optimizer,train_model_cv
from EEG_Tensorflow_models.Models import MTVAE

In [ ]:
import numpy as np
import pickle
from scipy.signal import decimate,resample

In [ ]:
path_ = '../input/gigascience-2017/'

In [ ]:
subjects = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25,
 26, 27, 28, 29, 30, 31, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 47, 48, 50, 51, 52]

In [ ]:
opt_args = {'lr': 0.001,'beta_1': 0.9}
optimizer = get_optimizer('Adam',opt_args)

loss = get_loss(['mse','CategoricalCrossentropy'])

metrics = ['accuracy']

callbacks_names = {'early_stopping_train1':'early_stopping','checkpoint_train1':'checkpoint',
                   'early_stopping_train2':'early_stopping','checkpoint_train2':'checkpoint',
                   'early_stopping_train3':'early_stopping','checkpoint_train3':'checkpoint',
                   'early_stopping_train4':'early_stopping','checkpoint_train4':'checkpoint'}


Experiment = 'lawhern2018'
model_name = 'DMTL_BCI'

#fs_new = 250

#subjects = np.arange(1,53)
Acc = []
History = []
Subject = []
for sbj in subjects:
    print('Subject: {:d} of {:d}'.format(sbj,len(subjects)))
    
    data = pickle.load( open( path_+'Subject_'+str(sbj)+'.p', "rb" ) )
    X_train = data['X'][:,:,0:-1,:]
    y_train = data['y']
    X_valid = data['X'][:,:,0:-1,:]
    y_valid = data['y']
    print(X_train.shape,y_train.shape)
    #X_train,y_train,X_valid,y_valid,fs = load_dataset(dataset_name="Cho2017", subject_id=sbj,split=False)
    #X_train = resample(X_train,int((X_train.shape[2]/fs)*fs_new),axis=2)[:,:,0:-1,:]
    #X_valid = resample(X_valid,int((X_valid.shape[2]/fs)*fs_new),axis=2)[:,:,0:-1,:]

    model_args = {'nb_classes':2,'Chans':X_train.shape[1],'Samples':X_train.shape[2],'dropoutRate':0.5}
    model = get_model(model_name,model_args)

    call_args = [
        {'monitor':'val_Classif_accuracy','patience':100,'min_delta':0.001,'mode':'max','verbose':1,'restore_best_weights':False},        
        {'filepath':Experiment+'checkpoint1_'+str(sbj),'save_format':'tf','monitor':'val_Classif_accuracy','verbose':1,'save_weights_only':True,'save_best_only':True},
        {'monitor':'val_Classif_accuracy','patience':100,'min_delta':0.001,'mode':'max','verbose':1,'restore_best_weights':False},
        {'filepath':Experiment+'checkpoint2_'+str(sbj),'save_format':'tf','monitor':'val_Classif_accuracy','verbose':1,'save_weights_only':True,'save_best_only':True},
        {'monitor':'val_Classif_accuracy','patience':100,'min_delta':0.001,'mode':'max','verbose':1,'restore_best_weights':False},
        {'filepath':Experiment+'checkpoint3_'+str(sbj),'save_format':'tf','monitor':'val_Classif_accuracy','verbose':1,'save_weights_only':True,'save_best_only':True},
        {'monitor':'val_Classif_accuracy','patience':100,'min_delta':0.001,'mode':'max','verbose':1,'restore_best_weights':False},
        {'filepath':Experiment+'checkpoint4_'+str(sbj),'save_format':'tf','monitor':'val_Classif_accuracy','verbose':1,'save_weights_only':True,'save_best_only':True},
    ]

    callbacks = get_callbacks(callbacks_names,call_args)

    cv = train_model_cv(model,optimizer,loss,metrics,callbacks=callbacks,loss_weights=[2.5,1])

    history = cv.fit_validation(X_train,y_train,X_val=X_valid,y_val=y_valid,batch_size=30,epochs=1000,verbose=1,val_mode=Experiment,autoencoder=True,early_stopping=True)
    acc = cv.get_accuracy()
    print('Subject accuracy: {:f}'.format(acc))
    Acc.append(acc)
    History.append(History)
    Subject.append(sbj)

    results = {}
    results['subject'] = Subject
    results['history'] = History
    results['accuracy'] = Acc

    with open('Results_BCI2a_'+Experiment+'_'+model_name+'.p','wb') as handle:
        pickle.dump(results,handle)

    del X_train,y_train,X_valid,y_valid
    